In [15]:
import math
import itertools
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine

In [4]:
data = pd.read_json('./data/jobs.json', orient='records')
data.job_description = data.job_description.apply(lambda t: t[0])
data

,date,job_description,salary,skills,title,url
0,2017-02-10,"<div class=""description"" itemprop=""description...","£55,000 per annum","[Oracle, SSRS, DBA, SQL Server]",Database Administrator,https://www.reed.co.uk/jobs/database-administr...
1,2017-02-09,"<div class=""description"" itemprop=""description...","£45,000 - £50,000 per annum",[],IT Manager - Clinical systems experience,https://www.reed.co.uk/jobs/it-manager-clinica...
2,2017-02-09,"<div class=""description"" itemprop=""description...",£250.00 - £300.00 per day,[],Senior Web / WinForms Developer - ASP.Net / C...,https://www.reed.co.uk/jobs/senior-web-winform...
3,2017-02-09,"<div class=""description"" itemprop=""description...","£35,000 - £45,000 per annum",[],JavaScript Developer (Angular),https://www.reed.co.uk/jobs/javascript-develop...
4,2017-02-09,"<div class=""description"" itemprop=""description...","£30,000 - £40,000 per annum","[ASP.NET, C#, MVC, Sitecore, Episerver]",Software Developer,https://www.reed.co.uk/jobs/software-developer...
5,2017-02-09,"<div class=""description"" itemprop=""description...",£290.00 - £300.00 per day,"[Oracle, Scrum, Testing, ISEB, STLC, agile, da...",Testers,https://www.reed.co.uk/jobs/testers/31626707
6,2017-02-09,"<div class=""description"" itemprop=""description...",£350.00 - £360.00 per day,"[Multitasking, QA and Testing, Meeting project...",Test Managers,https://www.reed.co.uk/jobs/test-managers/3162...
7,2017-02-09,"<div class=""description"" itemprop=""description...",£350.00 - £360.00 per day,"[Multitasking, QA and Testing, Meeting project...",Test Manager,https://www.reed.co.uk/jobs/test-manager/31626747
8,2017-02-09,"<div class=""description"" itemprop=""description...","£45,000 - £55,000 per annum","[Linux, Redis, AWS, Vagrant, DevOps, Laravel, ...",DevOps,https://www.reed.co.uk/jobs/devops/31626762
9,2017-02-09,"<div class=""description"" itemprop=""description...","£65,000 - £70,000 per annum, negotiable, pro-r...","[Applications, Project Management, Technology]",BI Project Manager,https://www.reed.co.uk/jobs/bi-project-manager...


In [5]:
# full_skills_doc = data[['job_description', 'skills']]
# full_skills_doc

# Term-Document Matrix

In [6]:
all_skills = data.skills.tolist()
all_skills = np.asarray(all_skills)

all_skills = list(itertools.chain(*all_skills))
all_skills[:5]

[u'Oracle', u'SSRS', u'DBA', u'SQL Server', u'ASP.NET']

In [7]:
td_matrix = pd.DataFrame()
td_matrix['skills'] = all_skills
td_matrix.drop_duplicates(inplace=True)

In [8]:
def term_frequency(t, d):
    return d.count(t)

In [9]:
idf_values = {}
all_skills = td_matrix['skills'].tolist()
num_of_docs = len(data.index)

for skill in all_skills:
    _skill = skill
    contains_token = map(lambda doc: _skill in doc, data.skills.tolist())
    idf_values[skill] = math.log(float(num_of_docs) / (1 + sum(contains_token)))

In [10]:
idf_values.get('SSRS')

5.809142990314028

In [13]:
print len(td_matrix)
print len(data)

1114
1000


In [14]:
# td_matrix = td_matrix
# data = data

def calc_td_matrix(i, row):
    for ix, tdrow in td_matrix.iterrows():
        doc = 'd' + str(i)
        td_matrix.loc[ix, doc] = idf_values.get(tdrow['skills'] ,0) * term_frequency(tdrow['skills'], row['job_description'])
    
for i, row in data.iterrows(): 
    calc_td_matrix(i, row)

# Export 
td_matrix.to_csv('tmp/td_matrix.csv', index=False, encoding='utf-8')

td_matrix

,skills,d0,d1,d2,d3,d4,d5,d6,d7,d8,...,d990,d991,d992,d993,d994,d995,d996,d997,d998,d999
0,Oracle,8.845697,0.0,0.000000,0.0,0.000000,8.845697,0.000000,0.000000,0.0,...,0.0,4.422849,0.0,0.000000,0.000000,0.000000,0.0,0.000000,13.268546,0.0
1,SSRS,11.618286,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,5.809143,0.0,0.000000,0.000000,0.0
2,DBA,10.596635,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
3,SQL Server,15.347987,0.0,5.115996,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,5.115996,0.0,0.000000,0.000000,15.347987,0.0,0.000000,0.000000,0.0
4,ASP.NET,0.000000,0.0,0.000000,0.0,17.074792,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,8.537396,0.0,0.000000,0.000000,0.0
5,C#,0.000000,0.0,23.350170,0.0,17.512627,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,2.918771,0.0,0.000000,0.000000,0.0
6,MVC,0.000000,0.0,3.649659,0.0,25.547611,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
7,Sitecore,0.000000,0.0,0.000000,0.0,40.664001,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
8,Episerver,0.000000,0.0,0.000000,0.0,43.502257,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
10,Scrum,0.000000,0.0,0.000000,0.0,0.000000,4.605170,0.000000,0.000000,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,4.605170,0.0


In [16]:
_td_matrix = td_matrix
_td_matrix = _td_matrix.set_index('skills')

skill_1 = _td_matrix.loc['Oracle'].tolist()
skill_2 = _td_matrix.loc['DBA'].tolist()

skills_sparse = sparse.csr_matrix(_td_matrix)
similarities = cosine_similarity(skills_sparse)
print('pairwise dense output:\n {}\n'.format(similarities))

pairwise dense output:
 [[ 1.          0.02756327  0.43179642 ...,  0.          0.          0.        ]
 [ 0.02756327  1.          0.15135887 ...,  0.1490712   0.          0.        ]
 [ 0.43179642  0.15135887  1.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.1490712   0.         ...,  1.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          1.          0.9486833 ]
 [ 0.          0.          0.         ...,  0.          0.9486833   1.        ]]



In [17]:
distance_matrix = pairwise_distances(skills_sparse, metric="cosine")
distance_matrix

array([[  2.33146835e-15,   9.72436726e-01,   5.68203584e-01, ...,
          1.00000000e+00,   1.00000000e+00,   1.00000000e+00],
       [  9.72436726e-01,   0.00000000e+00,   8.48641130e-01, ...,
          8.50928802e-01,   1.00000000e+00,   1.00000000e+00],
       [  5.68203584e-01,   8.48641130e-01,   2.22044605e-16, ...,
          1.00000000e+00,   1.00000000e+00,   1.00000000e+00],
       ..., 
       [  1.00000000e+00,   8.50928802e-01,   1.00000000e+00, ...,
          0.00000000e+00,   1.00000000e+00,   1.00000000e+00],
       [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00, ...,
          1.00000000e+00,   1.11022302e-16,   5.13167019e-02],
       [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00, ...,
          1.00000000e+00,   5.13167019e-02,   0.00000000e+00]])

In [20]:
x = pd.DataFrame(similarities)

In [21]:
x.to_csv('tmp/x.csv', index=False)

In [28]:
x.columns = _td_matrix.index
x.set_index(_td_matrix.index, inplace=True)

In [29]:
x

skills,Oracle,SSRS,DBA,SQL Server,ASP.NET,C#,MVC,Sitecore,Episerver,Scrum,...,IT Project Manager,digitising,digitizing,Code,Macintosh,Filemaker,IT Support Engineer / 2nd /3rd line Support Analyst /Infrastructure Support Engineer / IT Support Analyst/ IT Field Engineer / Onsite Support / Win,Software Tester / QA Engineer / Test Analyst / Manual / Automation / SQL Server / T-SQL / BI / Business Intelligence / Excel / Web Applications / ASP.,Statistics,Predictive Modelling
skills,,,,,,,,,,,,,,,,,,,,,
Oracle,1.000000,0.027563,0.431796,0.256213,0.002496,0.016849,0.002010,0.000000,0.000000,0.020690,...,0.00,0.0,0.00,0.045596,0.0,0.0,0.0,0.000000,0.000000,0.000000
SSRS,0.027563,1.000000,0.151359,0.520480,0.048295,0.045977,0.032407,0.000000,0.000000,0.006066,...,0.00,0.0,0.00,0.000000,0.0,0.0,0.0,0.149071,0.000000,0.000000
DBA,0.431796,0.151359,1.000000,0.186116,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
SQL Server,0.256213,0.520480,0.186116,1.000000,0.353480,0.266768,0.290325,0.052164,0.000000,0.014207,...,0.00,0.0,0.00,0.086100,0.0,0.0,0.0,0.130931,0.000000,0.000000
ASP.NET,0.002496,0.048295,0.000000,0.353480,1.000000,0.555617,0.690196,0.132299,0.215980,0.057123,...,0.00,0.0,0.00,0.159781,0.0,0.0,0.0,0.107990,0.000000,0.000000
C#,0.016849,0.045977,0.000000,0.266768,0.555617,1.000000,0.603440,0.212805,0.168232,0.043353,...,0.00,0.0,0.00,0.101409,0.0,0.0,0.0,0.028039,0.000000,0.000000
MVC,0.002010,0.032407,0.000000,0.290325,0.690196,0.603440,1.000000,0.207866,0.304348,0.053073,...,0.00,0.0,0.00,0.114365,0.0,0.0,0.0,0.000000,0.000000,0.000000
Sitecore,0.000000,0.000000,0.000000,0.052164,0.132299,0.212805,0.207866,1.000000,0.418330,0.012158,...,0.00,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
Episerver,0.000000,0.000000,0.000000,0.000000,0.215980,0.168232,0.304348,0.418330,1.000000,0.000000,...,0.00,0.0,0.00,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [44]:
x[(x >= 0.9).any(axis=1)].to_csv('./tmp/related_test.csv',encoding='utf8')